# Introduction to JumpStart - 画像分類

---
Amazon [SageMaker JumpStart](https://docs.aws.amazon.com/sagemaker/latest/dg/studio-jumpstart.html)へようこそ! JumpStartを使用すると、SageMaker Studioのワンクリック、または[SageMaker JumpStart API](https://sagemaker.readthedocs.io/en/stable/overview.html#use-prebuilt-models-with-sagemaker-jumpstart)を通じて、多くの機械学習タスクを解決することができます。

このノートブックのデモでは、画像分類のためのJumpStart APIを使用する方法を紹介します。画像分類とは、画像をトレーニングデータセットのクラスラベルのいずれかに分類することです。 このデモでは、画像分類モデルの2つの使用例を紹介します。

* ImageNetデータセットを使って事前学習されたモデルを使用して、画像を分類する方法。[ImageNetLabels](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt)。
* 独自のデータセットを使って事前学習済みモデルをファインチューニングし、推論を実行する方法。

注：このノートブックは、Amazon SageMaker Studio with Python 3 (Data Science) kernel の ml.t3.medium インスタンスと、Amazon SageMaker Notebook インスタンス with conda_python3 kernel でテストされています。

---

1. [セットアップ](#1.-セットアップ)
2. [学習済みモデルを選択する](#2.-学習済みモデルを選択する)
3. [学習済みモデルで推論する](#3.-学習済みモデルで推論する)
    * [JumpStartのモデルを取得してエンドポイントをデプロイする](#3.1.-JumpStartのモデルを取得してエンドポイントをデプロイする)
    * [推論用のサンプル画像をダウンロードする](#3.2.-推論用のサンプル画像をダウンロードする)
    * [エンドポイントにクエリし、レスポンスをパースする](#3.3.-エンドポイントにクエリし、レスポンスをパースする)
    * [エンドポイントを削除する](#3.4.-エンドポイントを削除する)
4. [独自のデータセットを使って学習済みモデルをファインチューニングする](#4.-独自のデータセットを使って学習済みモデルをファインチューニングする)
    * [JumpStartの学習用アーティファクトを取得する](#4.1.-JumpStartの学習用アーティファクトを取得する)
    * [トレーニングのパラメータを設定する](#4.2.-トレーニングのパラメータを設定する)
    * [自動モデルチューニングを使って学習する](#AMT)
    * [学習を開始する](#4.4.-学習を開始する)
    * [ファインチューニングしたモデルをデプロイして推論する](#4.5.-ファインチューニングしたモデルをデプロイして推論する)
    * [ファインチューニング済みモデルを増分学習する](#4.6.-ファインチューニング済みモデルを増分学習する)

## 1. セットアップ
***
ノートブックを実行する前に、セットアップに必要ないくつかの初期化のステップがあります。このノートブックには、最新版のsagemakerとipywidgetsが必要です。
***

In [ ]:
!pip install sagemaker ipywidgets --upgrade --quiet

---

Amazon Sagemakerでトレーニングやホスティングを行うには、AWSのサービスを利用するための設定と認証が必要です。ここでは、SageMakerにアクセスできるAWSアカウントロールとして、現在のノートブックインスタンスに関連付けられた実行ロールを使用します。S3内のデータへのアクセスなど、権限を持つ必要があります。

---

In [ ]:
import sagemaker, boto3, json
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

## 2. 学習済みモデルを選択する
***
デフォルトのモデルで進めることも、次のセルを実行したときに生成されるドロップダウンから別のモデルを選択することもできます。JumpStart の全モデルのリストは、[JumpStart Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/pretrainedmodels.html) からも確認できます。
***

In [ ]:
model_id, model_version, = (
    "pytorch-ic-mobilenet-v2",
    "*",
)

***
[オプション] 別の JumpStart モデルを選択します。ここでは、jumpstart model_manifest ファイルを jumpstart s3 バケットからダウンロードし、すべての 画像分類モデルから選択して、推論用のモデルを選択します。
***

In [ ]:
import IPython
from ipywidgets import Dropdown

# JumpStartの model_manifest ファイルをダウンロードします
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

# manifestファイルから全ての画像分類モデルを選択します
ic_models_all_versions, ic_models = [
    model["model_id"] for model in model_list if "-ic-" in model["model_id"]
], []
[ic_models.append(model) for model in ic_models_all_versions if model not in ic_models]

# ユーザ選択のため、model-idsのドロップダウンリストを表示します
dropdown = Dropdown(
    options=ic_models,
    value=model_id,
    description="JumpStart Image Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(IPython.display.Markdown("## Select a JumpStart pre-trained model from the dropdown below"))
display(dropdown)

## 3. 学習済みモデルで推論する
***
JumpStartを使えば、独自のデータセットで最初にファインチューニングを行わなくても、事前学習済みモデルで推論を実行することができます。この例では、入力画像に対して、ImageNetデータセットの1000クラスのうちの1つのクラスラベルを予測します。
[ImageNetLabels](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt)。
***

### 3.1. JumpStartのモデルを取得してエンドポイントをデプロイする
***
事前学習済みモデルの deploy_image_uri, deploy_source_uri, base_model_uri を取得します。事前学習済みのベースモデルをホストするために、[`sagemaker.model.Model`](https://sagemaker.readthedocs.io/en/stable/api/inference/model.html) のインスタンスを作成し、デプロイします。
***

In [ ]:
from sagemaker import image_uris, model_uris, script_uris
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base

# model_version="*" は最新のモデルバージョンを取得します
infer_model_id, infer_model_version = dropdown.value, "*"

endpoint_name = name_from_base(f"jumpstart-example-{infer_model_id}")

inference_instance_type = "ml.m5.xlarge"

# 推論用Dockerコンテナを取得します
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=infer_model_id,
    model_version=infer_model_version,
    instance_type=inference_instance_type,
)
# 推論用スクリプトを取得します
deploy_source_uri = script_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, script_scope="inference"
)
# ベースモデルのuriを取得します
base_model_uri = model_uris.retrieve(
    model_id=infer_model_id, model_version=infer_model_version, model_scope="inference"
)
# SageMaker モデルのインスタンスを作成します。sagemaker APIを通して推論を実行するには、model をデプロイする際に、Predictorクラスを通す必要があります。
model = Model(
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    model_data=base_model_uri,
    entry_point="inference.py",
    role=aws_role,
    predictor_cls=Predictor,
    name=endpoint_name,
)
# モデルをデプロイします。
base_model_predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
)

### 3.2. 推論用のサンプル画像をダウンロードする
***
JumpStart S3バケットからサンプル画像をダウンロードします。
***

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
key_prefix = "inference-notebook-assets"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


images = {"img1.jpg": "cat.jpg", "img2.jpg": "dog.jpg"}
download_from_s3(images)

### 3.3. エンドポイントにクエリし、レスポンスをパースする
***
エンドポイントへの入力は、2 値化された 1 枚の画像です。エンドポイントからのレスポンスは、上位 k 個の予測したクラスラベルと、各クラスの確率のリストを含む dictionary です。
***

In [ ]:
from IPython.core.display import HTML


def predict_top_k_labels(probabilities, labels, k):
    topk_prediction_ids = sorted(
        range(len(probabilities)), key=lambda index: probabilities[index], reverse=True
    )[:k]
    topk_class_labels = ", ".join([labels[id] for id in topk_prediction_ids])
    return topk_class_labels


for image_filename in images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = base_model_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    labels, probabilities = model_predictions["labels"], model_predictions["probabilities"]
    top5_class_labels = predict_top_k_labels(probabilities, labels, 5)
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Top-5 predictions: {top5_class_labels} </figcaption>"
        )
    )

### 3.4. エンドポイントを削除する

In [ ]:
# SageMakerエンdポイントとアタッチされたリソースを削除します
base_model_predictor.delete_model()
base_model_predictor.delete_endpoint()

## 4. 独自のデータセットを使って学習済みモデルをファインチューニングする
***
前回は、事前に学習したモデルに対して推論を実行する方法を確認しました。次に、任意の数のクラスを持つカスタムデータセットに対してモデルをファインチューニングする方法について説明します。

ファインチューニングが可能なモデルは、利用可能な TensorFlow/PyTorch Hub の対応する特徴抽出モデルに、分類レイヤーを取り付けて、パラメータをランダムな値に初期化します。分類層の出力次元
は入力データ中のクラス数に基づいて決定されます。ファインチューニングステップでモデルのパラメータをファインチューニングします。その目的は、入力データに対する分類誤差を最小にすることです
。ファインチューニングによって得たモデルは、推論のためにデプロイすることができます。以下は、モデルへの入力のために教師データをどのようにフォーマットすべきかを示しています。

- **入力:** クラスの数と同じ数の子ディレクトリを持つディレクトリ。 
    - それぞれの子ディレクトリは .jpg フォオーマットで画像を含む必要があります。 
- **出力:** 推論用にデプロイできる学習したモデル。 
    - ラベルのマッピングファイルは同じS3バケットに保存されます。 

入力ディレクトリは以下のようになります。
学習画像が、バラ(roses)とタンポポ(dandelion)の2つのクラスをもつ場合、入力ディレクトリは以下のようになります。s3のパスは`s3://bucket_name/input_directory/`のようになります。
末尾の `/` は必須であることに注意してください。フォルダ名と 'roses', 'dandelion', .jpg ファイル名についてはは何でもかまいません。
学習済みモデルとともにs3バケットに保存されるラベルのマッピングファイルは、モデルが出力するクラス確率のリストのインデックスに、フォルダ名'roses'と'dandelion'をマッピングします。
このマッピングは、フォルダ名のアルファベット順に行われます。以下の例では、モデル出力リストのインデックス0は'dandelion'に対応し、インデックス1は'roses'に対応します。

    input_directory
        |--roses
            |--abc.jpg
            |--def.jpg
        |--dandelion
            |--ghi.jpg
            |--jkl.jpg

モデルのファインチューニングのために、デフォルトのデータセットとして、tf_flowers データセットを提供します。
tf_flowers は5種類の花の画像を含みます。
データセットは [TensorFlow](https://www.tensorflow.org/datasets/catalog/tf_flowers) からダウンロードできます。. 
[Apache 2.0 License](https://jumpstart-cache-prod-us-west-2.s3-us-west-2.amazonaws.com/licenses/Apache-License/LICENSE-2.0.txt).
Citation:
<sub><sup>
@ONLINE {tfflowers,
author = "The TensorFlow Team",
title = "Flowers",
month = "jan",
year = "2019",
url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }
</sup></sub> source: [TensorFlow Hub](model_url). 
***

### 4.1. JumpStartの学習用アーティファクトを取得する
***
ここでは、選択されたモデルについて、学習用Dockerコンテナ、学習アルゴリズムソース、事前学習済みベースモデル、アルゴリズムがデフォルト値で使う学習用ハイパーパラメータのPython dictionaryを取得しています。model_version="*"は、最新のモデルを取得することに注意してください。また、train_image_uriを取得するために、training_instance_typeを指定する必要があります。（訳注: ml.p3.2xlarge を使用するには、上限緩和申請が必要です。詳しくは[こちら](https://aws.amazon.com/jp/premiumsupport/knowledge-center/resourcelimitexceeded-sagemaker/)をご覧ください。）
***

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = dropdown.value, "*"
training_instance_type = "ml.p3.2xlarge"

# Docker imageを取得します
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# 学習スクリプトを取得します
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# ファインチューニングのため事前学習済みモデルのtarアーカイブを取得します
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

### 4.2. トレーニングのパラメータを設定する
***
さて、必要な設定はすべて終わったので、次は画像分類のモデルのファインチューニングを行います。まず、[``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)オブジェクトを作成します。このEstimatorにより，学習ジョブが起動されます．

学習には 2 種類のパラメータを設定する必要があります。

1つ目はトレーニングジョブのパラメータです．(i) 教師データのパス．これは、入力データが格納されているS3フォルダです、(ii) Output path: 学習の出力が格納されるS3フォルダ。(iii) 学習インスタンスタイプ: 学習を実行するマシンの種類を示します。通常、これらの学習にはGPUインスタンスを使用します。ここでは、正しいtrain_image_uriを取得するために、トレーニングインスタンスタイプの定義は前のステップで実施しました。

2つ目のパラメータは、アルゴリズム固有の学習用ハイパーパラメータです。
***

In [ ]:
# 有効なバケット内のサンプル教師画像
training_data_bucket = f"jumpstart-cache-prod-{aws_region}"
training_data_prefix = "training-datasets/tf_flowers/"

training_dataset_s3_path = f"s3://{training_data_bucket}/{training_data_prefix}"

output_bucket = sess.default_bucket()
output_prefix = "jumpstart-example-ic-training"

s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

***
アルゴリズム固有のハイパーパラメータについては、まずアルゴリズムがデフォルト値で受け入れるトレーニングハイパーパラメータのPython dictionaryを取得します。これは独自の値に上書きすることができます。
***

In [ ]:
from sagemaker import hyperparameters

# モデルのファインチューニングのためのデフォルトのハイパーパラメータを取得します
hyperparameters = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [オプション] デフォルトのハイパーパラメータを独自の値で上書きします
hyperparameters["epochs"] = "5"
print(hyperparameters)

### 4.3. 自動モデルチューニングを使って学習する ([HPO: hyper parameter optimization](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)) <a id='AMT'></a>
***
Amazon SageMaker の自動モデルチューニングは、ハイパーパラメータチューニングとも呼ばれ、指定したアルゴリズムとハイパーパラメータの範囲で、データセット上で多くのトレーニングジョブを実行することにより、モデルのベストなバージョンを見つけます。そして、選択したメトリクスによって測定された、最高のパフォーマンスを発揮するモデルになるハイパーパラメータ値を選択します。Amazon SageMaker のハイパーパラメータチューニング API と連携するために [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) オブジェクトを使用します。
***

In [ ]:
from sagemaker.tuner import ContinuousParameter

# チューニングとモデルの選択にAMT (Automatic Model Tuning)を利用するかどうか 
use_amt = True

# ベストなモデルを選択するための目標となるメトリクスを、フレームワークごとに定義します
metric_definitions_per_model = {
    "tensorflow": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
    "pytorch": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val Acc: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
}

# モデルがサポートするハイパーパラメータを選択し、最適なモデルを学習するために探索する値の範囲を設定できます (https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")
}

# 精度を向上する（学習時間も増加します）ため、AMT (Automatic Model Tuning) の学習ジョブの数を増やす
max_jobs = 6
# 学習時間を短縮するために、AMT (Automatic Model Tuning) の並列学習ジョブ（アカウントごとにリミットで制限せれます）を変更します
# もし、最大ジョブ数と並列学習ジョブ数が同一の場合、探索方法が Bayesian から Random に変更されます。
max_parallel_jobs = 2

### 4.4. 学習を開始する
***
必要なアセットをすべて含む Estimator オブジェクトを作成し、トレーニングジョブを起動します。
***

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

# SageMakerのEstimatorインスタンスを作成します
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=s3_output_location,
    base_job_name=training_job_name,
)

if use_amt:
    metric_definitions = next(
        value for key, value in metric_definitions_per_model.items() if model_id.startswith(key)
    )

    hp_tuner = HyperparameterTuner(
        ic_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # ベストなハイパーパラメータのための探索するために、SageMakerのチューニングジョブを開始します
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    # トレーニングデータのS3パスを渡して、SageMaker 学習ジョブを開始します
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

## 4.5. ファインチューニングしたモデルをデプロイして推論する
***
学習されたモデルはそれ自体では何もしてくれません。ここでは、そのモデルを使って推論を行いたいと思います。この例では、画像のクラスラベルを予測することを意味します。[3.学習済みモデルで推論する](#3.-学習済みモデルで推論する) と同じステップを踏んでいます。まず、エンドポイントをデプロイするための JumpStart のアーティファクトを取得することから始めます。base_predictorの代わりに、fine-tuningした`ic_estimator`をデプロイします。
***

In [ ]:
inference_instance_type = "ml.m5.xlarge"

# 推論用 Docker コンテナの uri を取得します
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# 推論用のスクリプト uri を取得します
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}-")

# SageMakerエンドポイントをデプロイするため前のステップのestimatorを使用します
finetuned_predictor = (hp_tuner if use_amt else ic_estimator).deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

---
次に、S3バケットからバラとヒマワリのサンプル画像をダウンロードし、推論を行います。

---

In [ ]:
s3_bucket = f"jumpstart-cache-prod-{aws_region}"
key_prefix = "training-datasets/tf_flowers"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


flower_images = {
    "img1.jpg": "roses/10503217854_e66a804309.jpg",
    "img2.jpg": "sunflowers/1008566138_6927679c8a.jpg",
}
download_from_s3(flower_images)

---
次に、ファインチューニングしたモデルに対してクエリを行い、そのレスポンスをパースして予測値を表示します。

---

In [ ]:
from IPython.core.display import HTML

for image_filename in flower_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = finetuned_predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )

---
次に、デプロイしたエンドポイントを削除します。

---

In [ ]:
# SageMakerエンドポイントとアタッチされたリソースを削除します
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

## 4.6. ファインチューニング済みモデルを増分学習する

***
増分学習は、前回の学習で考慮されなかったためにモデルのパフォーマンスの低下をもたらしたパターンを含む、拡張されたデータセットを使用して、新しいモデルを学習させることができます。既存のモデルのアーティファクトと拡張されたデータセットを使用して新しいモデルを学習することができます。増分学習は、ゼロからモデルを再トレーニングする必要がないため、時間とリソースの両方を節約することができます。

データセットの形式（クラスのセット）が同じであれば、どんなデータセット（新旧問わず）を使ってもかまいません。増分学習ステップは、前述のファインチューニングと似ていますが、次の違いがあります。ファインチューニングでは事前学習済みモデルから開始するのに対し、増分学習では既存のファインチューニング済みモデルから始めます。
***

In [ ]:
# 学習ジョブ名とアーティファクトのアウトプット場所をもとに、前のステップで学習したモデルを特定します
if use_amt:  # amtを利用している場合、ベストな学習ジョブを選択します
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hp_tuner._current_job_name
    )
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name

last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"

In [ ]:
incremental_train_output_prefix = "jumpstart-example-ic-incremental-training"

incremental_s3_output_location = f"s3://{output_bucket}/{incremental_train_output_prefix}/output"

incremental_training_job_name = name_from_base(f"jumpstart-example-{model_id}-incremental-training")

incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparameters,
    output_path=incremental_s3_output_location,
    base_job_name=incremental_training_job_name,
)

incremental_train_estimator.fit({"training": training_dataset_s3_path}, logs=True)

学習が完了したら、4.5 [ファインチューニングしたモデルをデプロイして推論する](#4.5.-ファインチューニングしたモデルをデプロイして推論する) と同じ手順でモデルをデプロイすることができます。